### SqueezeNet Introduction

[Reference1](https://blog.csdn.net/xbinworld/article/details/50897870)
[Reference2](https://zhuanlan.zhihu.com/p/49465950)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import keras.backend as K

from keras.models import Model
from keras.layers import Input, Flatten, Dropout, Concatenate, Activation
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D

from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.utils.data_utils import get_file

def _fire(x, filters, name="fire"):
    sq_filters, ex1_filters, ex2_filters = filters
    squeeze = Convolution2D(sq_filters, (1, 1), activation='relu', 
                            padding='same', name=name + "/squeeze1x1")(x)
    expand1 = Convolution2D(ex1_filters, (1, 1), activation='relu', 
                            padding='same', name=name + "/expand1x1")(squeeze)
    expand2 = Convolution2D(ex2_filters, (3, 3), activation='relu', 
                            padding='same', name=name + "/expand3x3")(squeeze)
    x = Concatenate(axis=-1, name=name)([expand1, expand2])
    return x

def SqueezeNet(include_top=True, weights="imagenet", input_tensor=None, input_shape=None, 
               pooling=None, classes=1000):

    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),require_flatten=True)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = Convolution2D(64, kernel_size=(3, 3), strides=(2, 2), padding="same", 
                      activation="relu", name='conv1')(img_input)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool1', padding="valid")(x)

    x = _fire(x, (16, 64, 64), name="fire2")
    x = _fire(x, (16, 64, 64), name="fire3")

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool3', padding="valid")(x)

    x = _fire(x, (32, 128, 128), name="fire4")
    x = _fire(x, (32, 128, 128), name="fire5")

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool5', padding="valid")(x)

    x = _fire(x, (48, 192, 192), name="fire6")
    x = _fire(x, (48, 192, 192), name="fire7")

    x = _fire(x, (64, 256, 256), name="fire8")
    x = _fire(x, (64, 256, 256), name="fire9")

    if include_top:
        x = Dropout(0.5, name='dropout9')(x)

        x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
        x = AveragePooling2D(pool_size=(13, 13), name='avgpool10')(x)
        x = Flatten(name='flatten10')(x)
        x = Activation("softmax", name='softmax')(x)
    else:
        if pooling == "avg":
            x = GlobalAveragePooling2D(name="avgpool10")(x)
        else:
            x = GlobalMaxPooling2D(name="maxpool10")(x)

    model = Model(img_input, x, name="squeezenet")

    if weights == 'imagenet':
        model.load_weights('squeezenet_weights.h5')

    return model

Using TensorFlow backend.


In [2]:
model = SqueezeNet(weights='imagenet')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [3]:
W = K.eval(model.weights[0])
print(W.shape)

(3, 3, 3, 64)


### Load Sample Images

In [4]:
import os
import PIL
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import pandas as pd
import glob

In [5]:
def get_image_array(img_path, model, feature_column_names):
    label = img_path.split("/")[-2]
    filename = img_path.split("/")[-1]
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    pd_tmp = pd.Series(preds[0], index = feature_column_names)
    pd_tmp['file'] = img_path
    pd_tmp['label'] = int(label)
    return pd_tmp

In [6]:
workbookDir = os.getcwd()
image_path = os.path.join(workbookDir,"..", "data", "mnist","trainingSample", "*", "*")
image_files = [f for f in glob.glob(image_path) if f.endswith('.jpg')]
feature_column_names = [ "feature_" + str(i+1).zfill(4) for i in range(1000)]

In [7]:
df = pd.concat([get_image_array(img, model, feature_column_names) for img in image_files], axis =1).T
df.head(3)

,feature_0001,feature_0002,feature_0003,feature_0004,feature_0005,feature_0006,feature_0007,feature_0008,feature_0009,feature_0010,...,feature_0993,feature_0994,feature_0995,feature_0996,feature_0997,feature_0998,feature_0999,feature_1000,file,label
0,4.66226e-08,7.55054e-09,7.04572e-07,1.34009e-07,1.61186e-06,1.23846e-05,1.09417e-07,2.93435e-05,1.09632e-06,4.65919e-06,...,4.65366e-07,7.54915e-10,1.88382e-08,2.71086e-08,1.05278e-07,2.74433e-08,2.8624e-06,0.00231984,/home/pengtan/Projects/python/image-classifica...,2
1,2.39302e-08,1.88359e-08,1.93053e-07,2.32602e-08,3.54509e-07,1.76171e-05,1.67266e-07,2.57043e-05,1.51736e-06,8.60904e-06,...,5.79855e-07,4.1132e-09,6.86857e-08,5.36549e-08,1.26621e-07,9.50255e-08,1.11057e-06,0.000373204,/home/pengtan/Projects/python/image-classifica...,2
2,9.05708e-09,5.33913e-09,4.919e-08,7.30271e-09,1.78298e-07,2.01232e-06,1.80452e-08,1.60379e-05,8.53864e-07,1.1399e-06,...,1.06779e-07,1.08037e-09,1.64392e-08,8.9871e-09,5.13392e-08,1.78958e-08,5.09389e-07,0.000576674,/home/pengtan/Projects/python/image-classifica...,2


In [11]:
from PIL import Image
import IPython.display
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from skimage.measure import block_reduce
from sklearn.linear_model import LogisticRegression

In [13]:
X = df.filter(regex='feature_').values
y = df['label'].astype("str")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

SVC_model = SVC()
KNN_model = KNeighborsClassifier(n_neighbors=3)
RF_model = RandomForestClassifier(max_depth=10, random_state=0)

SVC_model.fit(X_train, y_train)
KNN_model.fit(X_train, y_train)
RF_model.fit(X_train, y_train)
clf = LogisticRegression(random_state=0).fit(X, y)

SVC_prediction = SVC_model.predict(X_test)
KNN_prediction = KNN_model.predict(X_test)
RF_prediction = RF_model.predict(X_test)
logit_prediction = clf.predict(X_test)

print(accuracy_score(SVC_prediction, y_test))
print(accuracy_score(KNN_prediction, y_test))
print(accuracy_score(RF_prediction, y_test))
print(accuracy_score(logit_prediction, y_test))

print(classification_report(SVC_prediction, y_test))
print(classification_report(KNN_prediction, y_test))
print(classification_report(RF_prediction, y_test))

0.3
0.26666666666666666
0.6083333333333333
0.38333333333333336
              precision    recall  f1-score   support

           0       0.88      0.28      0.42        25
           1       0.82      0.74      0.78        19
           2       0.06      1.00      0.12         1
           3       0.44      0.25      0.32        16
           4       0.07      0.50      0.12         2
           5       0.00      0.00      0.00        11
           6       0.08      0.12      0.10         8
           7       0.00      0.00      0.00         9
           8       0.13      0.13      0.13        15
           9       0.60      0.43      0.50        14

    accuracy                           0.30       120
   macro avg       0.31      0.35      0.25       120
weighted avg       0.47      0.30      0.34       120

              precision    recall  f1-score   support

           0       0.50      0.17      0.25        24
           1       0.76      0.68      0.72        19
           2   